In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_48187/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/11 07:59:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/11 07:59:51 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [15]:
data = [[1, '2015-01-01', 10], [2, '2015-01-02', 25],
        [3, '2015-01-03', 20], [4, '2015-01-04', 30], [5, '2015-01-06', 31], [6, '2015-01-07', 32]]
weather = pd.DataFrame(data, columns=['id', 'recordDate', 'temperature']).astype(
    {'id': 'Int64', 'recordDate': 'datetime64[ns]', 'temperature': 'Int64'})


df_person = spark.createDataFrame(weather)
# Register the DataFrame as a temporary view
df_person.createOrReplaceTempView("Weather")

In [18]:
df_person.show()

+---+-------------------+-----------+
| id|         recordDate|temperature|
+---+-------------------+-----------+
|  1|2015-01-01 00:00:00|         10|
|  2|2015-01-02 00:00:00|         25|
|  3|2015-01-03 00:00:00|         20|
|  4|2015-01-04 00:00:00|         30|
|  5|2015-01-06 00:00:00|         31|
|  6|2015-01-07 00:00:00|         32|
+---+-------------------+-----------+



In [25]:
query = """
	
	select
		wn.id
	from
		Weather wn
		left join Weather wp
		on date_sub(wn.recordDate, 1) = wp.recordDate
	where
		wn.temperature > wp.temperature

"""

In [26]:
spark.sql(query).show()

+---+
| id|
+---+
|  2|
|  4|
|  6|
+---+

